# Load Olist Data into MySQL

This notebook loads the Olist E-commerce dataset CSV files into a MySQL database.

## Prerequisites
1.  Ensure you have a MySQL server running.
2.  Create a database (e.g., `olist_db`) in MySQL Workbench.
3.  Fill in your database credentials below.

In [2]:
# Install necessary libraries if not already installed
!pip install pandas sqlalchemy pymysql mysql-connector-python


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sqlalchemy import create_engine
import os

# ==========================================
# CONFIGURATION - PLEASE UPDATE THESE VALUES
# ==========================================
DB_HOST = 'localhost'
DB_USER = 'root'        # Your MySQL username
DB_PASSWORD = 'peeyush*237' # Your MySQL password
DB_NAME = 'my_company'    # Your Database name (Create this in Workbench first!)

# Create the database connection string
# We use mysql+pymysql, but you can also use mysql+mysqlconnector
connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"

try:
    engine = create_engine(connection_string)
    # Test connection
    with engine.connect() as conn:
        print("Successfully connected to the database!")
except Exception as e:
    print(f"Error connecting to database: {e}")
    print("Please check your credentials and make sure the database exists.")

Successfully connected to the database!


In [3]:
# Define the path to your data directory
data_dir = os.path.join(os.getcwd(), 'small_data')

# List of files to ignore (if any)
ignore_files = []

def load_data_to_mysql(data_dir, engine):
    if not os.path.exists(data_dir):
        print(f"Data directory not found: {data_dir}")
        return

    files = [f for f in os.listdir(data_dir) if f.endswith('.csv') and f not in ignore_files]
    
    for filename in files:
        file_path = os.path.join(data_dir, filename)
        table_name = filename.replace('.csv', '').replace('olist_', '').replace('_dataset', '')
        
        print(f"Processing {filename} -> Table: {table_name}...")
        
        try:
            # Read CSV using chunks to avoid memory crashes with large files
            chunksize = 1000
            first_chunk = True
            
            for chunk in pd.read_csv(file_path, chunksize=chunksize):
                # Optional: specific column cleaning can go here
                
                # Write to SQL
                if_exists_action = 'replace' if first_chunk else 'append'
                chunk.to_sql(name=table_name, con=engine, if_exists=if_exists_action, index=False)
                first_chunk = False
                
            print(f"Successfully loaded {table_name}")
            
        except Exception as e:
            print(f"Failed to load {filename}: {e}")
            
load_data_to_mysql(data_dir, engine)

Processing olist_customers_dataset.csv -> Table: customers...
Failed to load olist_customers_dataset.csv: (pymysql.err.OperationalError) (3730, "Cannot drop table 'customers' referenced by a foreign key constraint 'orders_ibfk_1' on table 'orders'.")
[SQL: 
DROP TABLE customers]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Processing olist_geolocation_dataset.csv -> Table: geolocation...
Successfully loaded geolocation
Processing olist_orders_dataset.csv -> Table: orders...
Failed to load olist_orders_dataset.csv: (pymysql.err.OperationalError) (3730, "Cannot drop table 'orders' referenced by a foreign key constraint 'order_items_ibfk_1' on table 'order_items'.")
[SQL: 
DROP TABLE orders]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Processing olist_order_items_dataset.csv -> Table: order_items...
Successfully loaded order_items
Processing olist_order_payments_dataset.csv -> Table: order_payments...
Successfully loaded order_payments
Processing olist_ord

In [4]:
# Verification: Check if tables exist
try:
    from sqlalchemy import inspect
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    print("\nTables in database:")
    for table in tables:
        print(f"- {table}")
except Exception as e:
    print(f"Error verifying tables: {e}")


Tables in database:
- customers
- geolocation
- order_items
- order_payments
- order_reviews
- orders
- product_category_name_translation
- products
- sellers
